In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
from machinedesign.autoencoder.interface import load
import keras.backend as K
from keras.models import Model
import numpy as np
from skimage.io import imsave
from sklearn.neighbors import NearestNeighbors
from machinedesign.viz import grid_of_images_default
from sklearn.cluster import KMeans

In [79]:
job_id = '../mnist'

In [80]:
data = np.load('../results/jobs/{}/gen/generated.npz'.format(job_id))
X = data['generated']
clf = load('../../discr/digits')
clf_enc = Model(inputs=clf.layers[0].input, outputs=clf.get_layer('p_re_lu_4').output)
model = load('../results/jobs/{}'.format(job_id))
inp = model.layers[0].input
hid_layer = model.get_layer('winner_take_all_channel_1')
hid = hid_layer.output
out = model.layers[-1].output
enc = K.function([inp], hid)
dec = K.function([hid], out)

In [81]:
def F(x):
    h = enc([x])
    h = h.max(axis=(2, 3))
    h = h.reshape((h.shape[0], -1))
    return h

In [82]:
digits = np.load('../../data/digits.npz')['X'] / 255.0
htrue = F(digits[0:10000])
htrue_bin = (htrue > 0).astype(int)

In [72]:
knn = NearestNeighbors(n_neighbors=5, metric='hamming')
knn.fit(htrue_bin)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='hamming',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [73]:
x = X[0:10]
hfake = F(x)
hfake_bin = (hfake>0).astype(int)

In [78]:
dist, ind = knn.kneighbors(hfake_bin, return_distance=True)
dist *= hfake.shape[1]
print(dist[0:10])

[[ 0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.]]


In [75]:
nb = 20
xtrue = digits[ind[:, 0]][0:nb]
xfake = x[0:nb]
xall = np.concatenate((xfake, xtrue), axis=0)
im = grid_of_images_default(xall, shape=(2, len(xtrue)))
imsave('out.png', im)
print(xall.shape)

(20, 1, 28, 28)


/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [77]:
example = 0
f = hfake_bin[example]
t = htrue_bin[ind[example, 0]]
mask = (f==t)
mask = mask.astype(int)
nb = (((htrue_bin * mask) == (f * mask)).sum(axis=1) == f.shape[0]).sum()
print(nb)

3
